# Ejemplo de Quantization
---

En este ejemplo vamos a ver como cambiar la representación del modelo pasando los pesos y activaciones de FP32 a INT8. De esta forma, se obtinenen dos beneficios potenciales:


1.   Reducimos el tamaño que ocupa el modelo ya que los pesos ocupan una cuarta parte (8 bits vs 32 bits por peso).
2.   Si el dispositivo incorpora hardware para trabajar en 8 bits, se reduce el tiempo de ejecución. Sino, se mantiene el mismo que para 32 bits.

---

## 1. Instalar e importar las librerías necesarias

En este ejemplo vamos a trabajar con Pytorch y los modelos de torchvision

In [8]:
!pip3 install torchinfo onnx onnxruntime-gpu tensorrt onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 20.3 MB/s eta 0:00:00


In [2]:
from torchvision.models import alexnet, AlexNet_Weights
from torchinfo import summary
import torch
import torchvision
import time
import numpy as np

## 2. Definir el modelo

Definimos el modelo, en este caso, usamos AlexNet pre-entrenada en ImageNet. Usamos esta red ya que es una red lineal sin conexiones residuales que producen problemas con la cuantización. Este tipo de problemas se pueden solventar cambiando algunas operaciones del modelo como se ve en este ejemplo para ResNet50 (https://github.com/zanvari/resnet50-quantization/blob/main/quantization-resnet50.ipynb).

In [3]:
model = alexnet(AlexNet_Weights)
preprocessing = AlexNet_Weights.IMAGENET1K_V1.transforms()
summary(model, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "mult_adds"])

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 183MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
AlexNet                                  [1, 3, 224, 224]          [1, 1000]                 --                        --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 256, 6, 6]            --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 55, 55]           23,296                    70,470,400
│    └─ReLU: 2-2                         [1, 64, 55, 55]           [1, 64, 55, 55]           --                        --
│    └─MaxPool2d: 2-3                    [1, 64, 55, 55]           [1, 64, 27, 27]           --                        --
│    └─Conv2d: 2-4                       [1, 64, 27, 27]           [1, 192, 27, 27]          307,392                   224,088,768
│    └─ReLU: 2-5                         [1, 192, 27, 27]          [1, 192, 27, 27]          --                        --


## 3. Definir un data loader

Por limitaciones de tiempo de cómputo, vamos a trabajar con CIFAR-10 pero cualquier dataset es válido. Primero, tenemos que crear un DataLoader de Pytorch para poder usar los datos con nuestro modelo.

In [4]:
dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, transform=preprocessing, download=True)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

100%|██████████| 170M/170M [00:05<00:00, 30.7MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## 4. Preparar la cuantización

En este ejemplo vamos a usar una 'Quantization-Aware Training' para calibrar y transformar los pesos y activaciones de FP32 a INT8. De esta forma, los pesos se adaptan al nuevo rango de representación evitando problemas de cálculos que se salen fuera de rango y obteniendo un mejor accruacy que usando otras técnicas de cuantización como el 'Post-Training Quantization'.

Para ello, tenemos que añadir unos adaptadores a la entrada y salida del modelo para convertir las entradas de FP32 a INT8 y nuestras salidas de INT8 a FP32. Tras esto, definimos la librería que realizará la cuantización y que depende del hardware en el que vamos a desplegar. Pytorch ofrece las siguientes opciones: https://pytorch.org/docs/stable/quantization.html#backend-hardware-support

In [25]:
model_fp32 = torch.nn.Sequential(torch.ao.quantization.QuantStub(), model, torch.ao.quantization.DeQuantStub())
model_fp32.train()
model_fp32.qconfig = torch.ao.quantization.get_default_qat_qconfig('fbgemm')
model_fp32_prepared = torch.ao.quantization.prepare_qat(model_fp32)
summary(model_fp32_prepared, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "mult_adds"])

/tmp/ipython-input-4148835348.py:4: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_fp32_prepared = torch.ao.quantization.prepare_qat(model_fp32)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a fut

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
Sequential                                                   [1, 3, 224, 224]          [1, 1000]                 --                        --
├─QuantStub: 1-1                                             [1, 3, 224, 224]          [1, 3, 224, 224]          --                        --
│    └─FusedMovingAvgObsFakeQuantize: 2-1                    [1, 3, 224, 224]          [1, 3, 224, 224]          --                        --
├─AlexNet: 1-2                                               [1, 3, 224, 224]          [1, 1000]                 --                        --
│    └─Sequential: 2-2                                       [1, 3, 224, 224]          [1, 256, 6, 6]            --                        --
│    │    └─Conv2d: 3-1                                      [1, 3, 224, 224]          [1, 64, 55, 55]           23,296                    0


## 5. Entrenamiento del modelo

Realizamos unas épocas para calibrar los pesos del modelo y adaptarlo a la nueva representación. Para ello, usamos la base de datos que hemos descargado en el punto 3.

In [26]:
n_epochs = 1
opt = torch.optim.Adam(model_fp32_prepared.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
model_fp32_prepared.train().to('cuda')
for epoch in range(n_epochs): # Entrenamos n epocas
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    time_start = time.time()
    for inputs, labels in train_data_loader: # Obtenemos todos los batch de entrenamiento y los usamos para entrenar
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        opt.zero_grad()
        outs = model_fp32_prepared(inputs)
        loss = loss_fn(outs, labels)
        train_running_loss += loss.item()
        _, preds = torch.max(outs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        counter = counter + 1
        loss.backward()
        opt.step()

    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(train_data_loader.dataset))
    time_end = time.time() - time_start
    print(f'** Summary for epoch {epoch}: '
		f'loss: {epoch_loss:#.3g}, acc: {epoch_acc:#.3g}]  '
		f'time: {time_end:.3f}s **')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


** Summary for epoch 0: loss: 1.17e+07, acc: 9.99]  time: 121.802s **


## 6. Exportar el modelo en INT8

Una vez que hemos realizado el entrenamiento para pasar a INT8, simplemente limpiamos las capas auxiliares que añade Pytorch para realizar la calibración y exportamos el modelo a TorchScript para poder usarlo en un móvil.

In [27]:
# Es crucial poner el modelo en modo de evaluación y en la CPU
model_fp32_prepared.to('cpu').eval()
model_int8 = torch.ao.quantization.convert(model_fp32_prepared, inplace=False)

print("Exportando el modelo preparado para QAT a ONNX...")
dummy_input = torch.randn(1, 3, 224, 224, device='cpu')
onnx_model_path = "alexnet_qat_int8.onnx"

torch.onnx.export(
    model_int8,
    dummy_input,
    onnx_model_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    fallback=True
)
print(f"Modelo exportado a {onnx_model_path} exitosamente.")

/tmp/ipython-input-3856809601.py:3: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = torch.ao.quantization.convert(model_fp32_prepared, inplace=False)
W0219 18:55:14.036000 374 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 13 is a lower version than we have implementations 

Exportando el modelo preparado para QAT a ONNX...


W0219 18:55:14.606000 374 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0219 18:55:14.608000 374 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0219 18:55:14.610000 374 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.
W0219 18:55:14.613000 374 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.


[torch.onnx] Obtain model graph for `Sequential([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Sequential([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `Sequential([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `Sequential([...]` with `torch.export.export(..., strict=True)`... ❌
[torch.onnx] Falling back to legacy torch.onnx.export due to the following error: Failed to export the model with torch.export. This is step 1/3 of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the *torch.export* component and attach the full error stack as well as reproduction scripts.

## Exception summary

<cla

Además, vamos a realizar una  inferencia de prueba para analizar el rendimiento del modelo inicial y el cuantizado.

In [ ]:
import onnxruntime as ort
import numpy as np
import time

ONNX_FILE_PATH = "alexnet_qat_int8.onnx"
BATCH_SIZE = 1

print(f"Versión de ONNX Runtime: {ort.get_device()}")
print(f"Proveedores disponibles: {ort.get_available_providers()}")

# ONNX Runtime intentará usarlos en el orden especificado.
# 'TensorrtExecutionProvider' usará TensorRT para la máxima aceleración en INT8.
# 'CUDAExecutionProvider' es un fallback que también usa la GPU.
# 'CPUExecutionProvider' se usará si los otros fallan.
providers = [
    'TensorrtExecutionProvider',
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

print(f"\nCreando sesión de inferencia con los proveedores: {providers}")
session = ort.InferenceSession(ONNX_FILE_PATH, providers=providers)

# Obtener los nombres de entrada y salida del modelo
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
input_shape = session.get_inputs()[0].shape
dummy_preprocessed_image = np.random.rand(BATCH_SIZE, 3, 224, 224).astype(np.float32)

print("\nEjecutando inferencia...")
start_time = time.time()

# La inferencia se ejecuta con una simple llamada a 'run()'
results = session.run([output_name], {input_name: dummy_preprocessed_image})[0]

end_time = time.time()
print(f"Inferencia completada en {end_time - start_time:.4f} segundos.")

import io

def get_model_size_in_memory(model):
    """
    Calcula el tamaño del state_dict del modelo en un buffer de memoria.
    """
    # Creamos un buffer en memoria
    buffer = io.BytesIO()
    # Guardamos el diccionario de estado del modelo en el buffer
    torch.save(model.state_dict(), buffer)
    # Obtenemos el tamaño del buffer en bytes
    size_in_bytes = buffer.getbuffer().nbytes
    return size_in_bytes

print("\n--- Calculando el tamaño de los modelos en memoria ---")

# 1. Medir el modelo original (FP32)
fp32_size = get_model_size_in_memory(model)
print(f"Modelo Original (FP32): {fp32_size / 1e6:.2f} MB")

# 2. Medir el modelo cuantizado (INT8)
# Asegúrate de que tienes la variable 'model_int8' o similar
int8_size = get_model_size_in_memory(model_int8)
print(f"Modelo Cuantizado (INT8): {int8_size / 1e6:.2f} MB")

# 3. Calcular y mostrar la reducción
reduction = 100 * (1 - int8_size / fp32_size)
print(f"Reducción de tamaño: {reduction:.2f}%")

torch.Size([1, 3, 224, 244])
Execution time of the fp32 model: 0.129s
Execution time of the int8 model: 0.035s
